In [1]:
import pandas as pd
# from sklearn.ensemble
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

def readFile(filename):
    f = pd.read_csv(f'./_data/{filename}.csv')
    return f

def ACCURACY(true, pred):
    score = np.mean(true==pred)
    return score

def z_oForType(data,columnName):
    columns = set(data[columnName].values)
    colNames = {}
    for key, value in enumerate(columns):
        colNames[value] = key
    data[columnName] = data[columnName].map(lambda x: colNames[x])
    #data[columnName] = data[columnName].replace(colNames) 로 해도 된답니다.
    return data

In [2]:
train_data = readFile('train')      #[3231 rows x 14 columns]
train_data = z_oForType(train_data, 'type')

#print(train_data)
test_data = readFile('test')
test_data = z_oForType(test_data, 'type')

# 결측치 확인
print(f"train_data의 결측치:")
print(f"{train_data.isna().sum()}")
print("-"*30)
print(f"test_data 결측치:")
print(f"{test_data.isna().sum()}")
print("-"*30)

train_data의 결측치:
id                      0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
type                    0
quality                 0
dtype: int64
------------------------------
test_data 결측치:
id                      0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
type                    0
dtype: int64
------------------------------


In [36]:
X_train = train_data.drop(columns=['id', 'quality'])
y_train = train_data['quality']

X_test = test_data.drop(columns=['id'])

# 데이터 불균형 해결을 위해 샘플링
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [37]:
X_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,8.0,0.18,0.37,1.30,0.040,15.0,96.0,0.99120,3.06,0.61,12.1,1
1,7.5,0.38,0.29,4.90,0.021,38.0,113.0,0.99026,3.08,0.48,13.0,1
2,6.1,0.27,0.33,2.20,0.021,26.0,117.0,0.98860,3.12,0.30,12.5,1
3,6.4,0.32,0.50,10.70,0.047,57.0,206.0,0.99680,3.08,0.60,9.4,1
4,8.4,0.37,0.43,2.30,0.063,12.0,19.0,0.99550,3.17,0.81,11.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3226,6.4,0.16,0.31,5.30,0.043,42.0,157.0,0.99455,3.35,0.47,10.5,1
3227,6.4,0.18,0.28,17.05,0.047,53.0,139.0,0.99724,3.25,0.35,10.5,1
3228,6.0,0.28,0.25,1.80,0.042,8.0,108.0,0.99290,3.08,0.55,9.0,1
3229,7.7,0.28,0.29,6.90,0.041,29.0,163.0,0.99520,3.44,0.60,10.5,1


In [ ]:
# 랜덤포레스트에서 뽑은 특성 중요도
# 5       alcohol       0.21955641097862588
#  3       total sulfur dioxide       0.20417962924895824
#  2       free sulfur dioxide       0.19582353804919067
#  1       volatile acidity       0.19278710748351738
#  4       density       0.18765331423970777

In [7]:
X_train = X_train.iloc[:,[-2,-6,-7,1,-5]]
# 주요특성 5개만 이용
sm = SMOTE(random_state=42)
X_res5, y_res5 = sm.fit_resample(X_train, y_train)

In [8]:
from sklearn.model_selection import train_test_split
X_train_resample,  X_test_resample, y_train_resample, y_test_resample = train_test_split(X_res5, y_res5, test_size=0.3, random_state=1)

In [23]:
#커널SVM
from sklearn.svm import SVC
svc =  SVC(kernel='rbf', random_state=1, gamma=0.3, C=2.0)
svc.fit(X_train_resample, y_train_resample)
prediction = svc.predict(X_test_resample)
print(ACCURACY(prediction, y_test_resample))

0.6718382698636577


In [ ]:
# svc 파라미터 https://chrisalbon.com/code/machine_learning/support_vector_machines/svc_parameters_using_rbf_kernel/

In [31]:
# 표준화를 안했다..
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
pipe_svc = make_pipeline(StandardScaler(), svc)
pipe_svc.fit(X_train_resample,y_train_resample)
prediction = pipe_svc.predict(X_test_resample)
print(ACCURACY(prediction, y_test_resample))

0.5665256229431124


In [24]:
#그리드서치로 하이퍼파라미터튜닝
from sklearn.model_selection import GridSearchCV
# refit =True (default) best모델 자동훈련
pipe_svc = make_pipeline(StandardScaler(), svc)
param_range = [0.1,1.0,10.0,100.0,1000.0,10000.0]
param_grid = [{'svc__C': param_range,'svc__kernel':['linear']},
             {'svc__C':param_range, 'svc__gamma':param_range, 'svc__kernel':['rbf']}]
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=10,refit=True,n_jobs=-1)
gs = gs.fit(X_train_resample,y_train_resample)
print(gs.best_params_)
# 너무 오래걸림 ㅜ 랜덤서치로 바꿔보기

{'svc__C': 10.0, 'svc__gamma': 10.0, 'svc__kernel': 'rbf'}


In [30]:
#gs.best_estimator_.fit(X_train_resample, y_train_resample)
print(gs.best_estimator_.score(X_test_resample,y_test_resample))

0.7465914433474377


In [35]:
# 랜덤 서치
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.fixes import loguniform
distribution = loguniform(0.0001,1000.0)
param_dist = [{'svc__C': param_range,'svc__kernel':['linear']},
             {'svc__C':param_range, 'svc__gamma':param_range, 'svc__kernel':['rbf']}]
rs = RandomizedSearchCV(estimator=pipe_svc, param_distributions=param_dist, n_iter=30, cv=10, random_state=1, n_jobs=-1)
rs = rs.fit(X_train_resample,y_train_resample)
print(rs.best_params_)
print(rs.best_score_)

{'svc__kernel': 'rbf', 'svc__gamma': 10.0, 'svc__C': 10.0}
0.7358380119426235


In [25]:
# 시도해볼것
#1. 모델 바꾸기 :Xgboost로 해보기
#2. 데이터 바꾸기 : 샘플링 다시하기, 더 불려보거나..
#3. 모델을 여러개 만들기 : 품질별로 훈련..??